In [25]:
# Solve a generalized assignment problem between the different tracklets 
# setting similarity weights on the assignments such that we maximize the
# total assignment weigthed objective and match the most likely tracklets
from gurobipy import *
from numpy import *
import os
import fnmatch

In [90]:
print cwd + '/3_obj_less_noisy'

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj_less_noisy


In [150]:
import fnmatch
import os
cwd = os.getcwd()
matches = []
for root, dirnames, filenames in os.walk(cwd):
    for filename in fnmatch.filter(filenames, 'Pij_2.csv'):
        file_path = os.path.join(root, filename)
        P = loadtxt(open(root + '/' + filename, "rb"), delimiter=",")
#         if root == '/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj_obsolete/csv_3_1':
        print root
        print P
        numT = shape(P)[0]
        try:
            numC = shape(P)[1]
        except:
            numC = 0;    

        print numC
        T=range(numT)
        C=range(numC)

        # Initialize the model
        m=Model("Assignment")

        # Define the binary variables Xs
        X = []
        for t in T:
            X.append([])
            if numC != 0:
                for c in C:
                    X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, c)))
            else:
                X[t].append(m.addVar(vtype=GRB.BINARY,name="X%d%d"% (t, 0)))

        # Set the optimization direction
        m.update()
        m.modelSense = GRB.MAXIMIZE


        # Define the assignment constraints
        constraintT = []
        constraintC = []


        if numC != 0:
            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(quicksum(X[t][c] for c in range(numC))
                                               <= 1 ,'constraintT%d' % t))
            # One variable assigned per row
            for c in range(numC):
                constraintT.append(m.addConstr(quicksum(X[t][c] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))

            # Define objective
            m.setObjective(quicksum(quicksum([X[t][c]*P[t][c] 
                                    for c in range(numC)]) 
                                    for t in range(numT)))
        else:
            # One variable assigned per column
            for t in range(numT):
                constraintT.append(m.addConstr(X[t][0]<= 1 ,'constraintT%d' % t))  

            # One variable assigned per row
            constraintT.append(m.addConstr(quicksum(X[t][0] for t in range(numT))
                                               <= 1 ,'constraintC%d' % t))

            # Define objective                 
            m.setObjective(quicksum(X[t][0]*P[t] for t in range(numT)))



        m.update()

        # Solve the optimization problem
        m.optimize()

        # Print runtime and solution

        print 'runtime is',m.Runtime, 's'
        print m


        if numC != 0:
            Xsol = zeros((numT,numC))
        else:
            Xsol = zeros(numT)
        # Print solution
        if m.status == GRB.Status.OPTIMAL:
            for t in range(numT):
                if numC != 0:
                    for c in range(numC):
                        if X[t][c].x > 0.5:
                            print('%g -> %g' % (t, c))
                            Xsol[t][c] = 1
                else:
                    if X[t][0].x > 0.5:
                        print('0 -> %g' % (t))
                        Xsol[t] = 1
        print Xsol
        name_sol = root + "/Assignment.csv"
        if numC != 0:
            savetxt(name_sol, Xsol, delimiter=",")
        else:
            savetxt(name_sol, Xsol[None], delimiter=",")
        

/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_59
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -9.03330000e-01   1.36330000e+00]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02   3.63330000e-01  -9.03330000e-01]]
7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.72663

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1.72663 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.726630000000e+00, best bound 1.726630000000e+00, gap 0.0000%
runtime is 0.0180711746216 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars,

Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.06

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1.06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.060000000000e+00, best bound 1.060000000000e+00, gap 0.0000%
runtime is 0.0175259113312 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
1 -> 5
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_33
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -5.70000000e-01   8.63330000e-01  -1.43670000e+00]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9

Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.69667

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.69667 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.966700000000e-01, best bound 6.966700000000e-01, gap 0.0000%
runtime is 0.0170738697052 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_69
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -3.45000000e-01   1.03000000e+00]
 [ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.01 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e-02, best bound 1.000000000000e-02, gap 0.0000%
runtime is 0.0162651538849 s
<gurobi.Model MIP instance Assignment: 7 constrs, 6 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
[ 0.  0.  0.  0.  0.  1.]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_64
[[-999.99   -999.99   -999.99   -999.99   -999.99      1.03     -1.3064]
 [-999.99   -999.99   -999.99   -999.99   -999.99   -999.99      1.03  ]]
7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix ran

Variable types: 0 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.72

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.72 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.200000000000e-01, best bound 7.200000000000e-01, gap 0.0000%
runtime is 0.0182719230652 s
<gurobi.Model MIP instance Assignment: 10 constrs, 21 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
1 -> 6
[[ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj_less_noisy/csv_5_15
[[ -9.99990000e+02  -9.99990000e+02  -9.99990000e+02  -9.99990000e+02
   -9.99990000e+02  -3.98570000e-01  -3.98570000e-01   1.53000000e+00]
 [ -9.99990000e+02  -9.9

...

  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-05, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01
Presolve removed 9 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 0.060357 0.01 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.035700000000e-02, best bound 6.035700000000e-02, gap 0.0000%
runtime is 0.0221040248871 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj/csv_5_26
[ -9.99840000e+02  -9.99840000e+02  -9.99840000e+02   1.49640000e-01
   2.11110000e-02   1.00000000e-02]
0
Optimize a model with 7 rows, 6 columns and 12 nonzeros
Variable types: 0 con

Best objective 8.244500000000e-02, best bound 8.244500000000e-02, gap 0.0000%
runtime is 0.0166761875153 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
1 -> 5
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj/csv_5_95
[[ -9.99590000e+02  -9.99590000e+02  -9.99590000e+02  -9.99590000e+02
   -9.99590000e+02  -8.76660000e-03   9.23600000e-02]
 [ -9.99590000e+02  -9.99590000e+02  -9.99590000e+02  -9.99590000e+02
   -9.99590000e+02   1.00000000e-02  -2.49210000e-02]]
7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.10236

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count w

    4.19650000e-02]]
9
Optimize a model with 13 rows, 36 columns and 72 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.193799
Presolve removed 9 rows and 32 columns
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 8 nonzeros
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: objective 2.035280e-01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.2035280    0.20353  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.04 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 0.203528 0.193799 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.035280000

Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.01

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0.01 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e-02, best bound 1.000000000000e-02, gap 0.0000%
runtime is 0.0219030380249 s
<gurobi.Model MIP instance Assignment: 7 constrs, 6 vars, Parameter changes: LogFile=gurobi.log>
0 -> 5
[ 0.  0.  0.  0.  0.  1.]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj/csv_5_99
[ -9.99760000e+02  -9.99760000e+02  -9.99760000e+02  -9.99760000e+02
   1.00000000e-02   7.00370000e-02]
0
Optimize a model with 7 rows, 6 columns and 12 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range   

Best objective 8.809300000000e-02, best bound 8.809300000000e-02, gap 0.0000%
runtime is 0.0127010345459 s
<gurobi.Model MIP instance Assignment: 9 constrs, 14 vars, Parameter changes: LogFile=gurobi.log>
0 -> 6
1 -> 5
[[ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/5_obj/csv_5_18
[[ -9.99540000e+02  -9.99540000e+02  -9.99540000e+02  -9.99540000e+02
   -9.99540000e+02   1.00000000e-02   2.34320000e-01]
 [ -9.99540000e+02  -9.99540000e+02  -9.99540000e+02  -9.99540000e+02
   -9.99540000e+02  -9.99540000e+02   4.44330000e-02]]
7
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.054433
Presolve removed 9 rows and 14 columns
Presolve time: 0.00s
Presolve: 

Optimize a model with 10 rows, 25 columns and 50 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: -0 

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


runtime is 0.0195119380951 s
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
/home/david/Dropbox/Ubuntu/Convex_Optimization/Project/Experiments/3_obj/csv_3_22
